# Machine Learning - Classification

## Binary Classification for Churn Prediction

$$g(x_i) = y_i$$

**Where:**

- the result of $y_i$ will either be \{0, 1\} ---> between 0 - 1 as the likelihood of churn
- $x_i$ — feature vector for the i-th customer  
- 1 is positive (churn)
- 0 is negative (no churn)

## Data prep
- Download the data, read it with pandas
- Look at the data
- Make column names and values look uniform
- Check if all the columns read correctly
- Check if the churn variables needs any preparation

In [105]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [106]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/refs/heads/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [107]:
df = pd.read_csv(data)
len(df)

7043

In [108]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [109]:
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [110]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [111]:
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [112]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [113]:
df.totalcharges = pd.to_numeric(df['totalcharges'], errors='coerce')

In [114]:
df.totalcharges = df.totalcharges.fillna(0)

In [115]:
df.churn

0        no
1        no
2       yes
3        no
4       yes
       ... 
7038     no
7039     no
7040     no
7041    yes
7042     no
Name: churn, Length: 7043, dtype: object

In [116]:
# df.churn = df.churn.map({'yes': 1, 'no': 0}) # Convert to numerical

# df.churn = df.churn.apply(lambda x: 1 if x == 'yes' else 0) # Convert to numerical

# df.churn = df.churn.replace({'yes': 1, 'no': 0}) # Convert to numerical

df.churn = (df.churn == 'yes').astype(int)  # Convert to numerical - True/False to 1/0

# all these methods do the same thing

In [117]:
df.churn

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: churn, Length: 7043, dtype: int64

In [118]:
df.head(10).T

,0,1,2,3,4,5,6,7,8,9
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu,9305-cdskc,1452-kiovk,6713-okomc,7892-pookp,6388-tabgu
gender,female,male,male,male,female,female,male,female,female,male
seniorcitizen,0,0,0,0,0,0,0,0,0,0
partner,yes,no,no,no,no,no,no,no,yes,no
dependents,no,no,no,no,no,no,yes,no,no,yes
tenure,1,34,2,45,2,8,22,10,28,62
phoneservice,no,yes,yes,no,yes,yes,yes,no,yes,yes
multiplelines,no_phone_service,no,no,no_phone_service,no,yes,yes,no_phone_service,yes,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic,fiber_optic,fiber_optic,dsl,fiber_optic,dsl
onlinesecurity,no,yes,yes,yes,no,no,no,yes,no,yes


## Setting up the validation framework
- Perform the train/validation/test split with Scikit-Learn

In [119]:
from sklearn.model_selection import train_test_split

In [120]:
# train_test_split? # This is to check the function signature and docstring and how large the default test size is

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1) # 20% for testing, 80% for training

In [121]:
len(df_full_train), len(df_test)

(5634, 1409)

In [122]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2, so 20% for validation, 60% for training, 20% for testing

In [123]:
len(df_train), len(df_val), len(df_test)   

(4225, 1409, 1409)

In [124]:
df_train

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
3897,8015-ihcgw,female,0,yes,yes,72,yes,yes,fiber_optic,yes,...,yes,yes,yes,yes,two_year,yes,electronic_check,115.50,8425.15,0
1980,1960-uycnn,male,0,no,no,10,yes,yes,fiber_optic,no,...,yes,no,no,yes,month-to-month,yes,electronic_check,95.25,1021.55,0
6302,9250-wypll,female,0,no,no,5,yes,yes,fiber_optic,no,...,no,no,no,no,month-to-month,no,electronic_check,75.55,413.65,1
727,6786-obwqr,female,0,yes,yes,5,yes,no,fiber_optic,no,...,no,no,yes,no,month-to-month,yes,electronic_check,80.85,356.10,0
5104,1328-euzhc,female,0,yes,no,18,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,20.10,370.50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3774,1309-xgfsn,male,1,yes,yes,52,yes,yes,dsl,no,...,yes,no,yes,yes,one_year,yes,electronic_check,80.85,4079.55,0
6108,4819-hjpiw,male,0,no,no,18,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,no,mailed_check,25.15,476.80,0
1530,3703-vavcl,male,0,yes,yes,2,yes,no,fiber_optic,no,...,yes,yes,no,yes,month-to-month,no,credit_card_(automatic),90.00,190.05,1
3701,3812-lrzir,female,0,yes,yes,27,yes,yes,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,electronic_check,24.50,761.95,0


In [125]:
df_train =df_train.reset_index(drop=True)
df_val =df_val.reset_index(drop=True)
df_test =df_test.reset_index(drop=True)

In [126]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [127]:
del df_train['churn']
del df_val['churn'] 
del df_test['churn']

## EDA
- Check missing values
- Look at the target variable (churn)
- Look at numerical and categorical variables

In [128]:
df_full_train = df_full_train.reset_index(drop=True)
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [129]:
df_full_train.churn.value_counts()
df_full_train.churn.value_counts(normalize=True)

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

In [130]:
# in this binary dataset, the mean gives us the exact churn rate, exactly the same result as value_counts(normalize=True)
global_churn_rate = df_full_train.churn.mean()
print(round(global_churn_rate, 2)) # 27% of customers churn

0.27


In [131]:
df_full_train.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                 int64
dtype: object

In [132]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [133]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [134]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

### Feature importance: Churn rate and risk ratio
Feature importance analysis (part of EDA) - identifying which features affect our target variable
- Churn rate
- Risk ratio
- Mutual information - later

**Churn rate**

Churn rate within each group 

In [135]:
global_churn = df_full_train.churn.mean()
print('global churn rate:', global_churn)

global churn rate: 0.26996805111821087


In [136]:
churn_female =df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
print('female churn rate:', churn_female, '\n', 'male churn rate:', churn_male)

female churn rate: 0.27682403433476394 
 male churn rate: 0.2632135306553911


In [137]:
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()
print('churn rate with partner:', churn_partner, '\n', 'churn rate without partner:', churn_no_partner)

churn rate with partner: 0.20503330866025166 
 churn rate without partner: 0.3298090040927694


In [138]:
global_churn - churn_partner

np.float64(0.06493474245795922)

In [139]:
global_churn - churn_no_partner

np.float64(-0.05984095297455855)

In [140]:
global_churn - churn_female

np.float64(-0.006855983216553063)

In [141]:
global_churn - churn_male

np.float64(0.006754520462819769)

The churn rate difference is higher for the partner feature than for the gender feature. This suggests that the partner feature is a better predictor of churn than the gender feature. (Partner variable is more important for churn prediction than the gender variable)

for the differences:

$$Global - Group$$
- $<0$ ==> more likely to churn
- $>0$ ==> less likely to churn

**Risk ratio**

In [142]:
churn_no_partner / global_churn

np.float64(1.2216593879412643)

In [143]:
churn_partner / global_churn

np.float64(0.7594724924338315)

for the risk ratio:

$$Risk = \frac{Group}{Global}$$
- $>1$ ==> more likely to churn
- $<1$ ==> less likely to churn

for churn_no_partner group, the risk ratio is greater than 1, meaning they are more likely to churn compared to the global average.

for churn_partner group, the risk ratio is less than 1, meaning they are less likely to churn compared to the global average.

churn_no_partner ==> High risk

churn_partner ==> Low risk

```sql
SELECT
    gender,
    AVG(churn),
    AVG(churn) - global_churn AS diff,
    AVG(churn) / global_churn AS risk
FROM data
GROUP BY gender;
```

In [144]:
from IPython.display import display

In [145]:
for c in categorical:
    print(c)
    df_group =df_full_train.groupby(c).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print()
    print()

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208




partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


### Feature importance: Mutual information
Mutual information - concept from information theory, it tells us how much we can learn about one variable if we know the other value of another
- https://en.wikipedia.org/wiki/Mutual_information

In [146]:
from sklearn.metrics import mutual_info_score

In [148]:
mutual_info_score(df_full_train.churn, df_full_train.contract)

0.0983203874041556

In [150]:
mutual_info_score(df_full_train.gender, df_full_train.churn)

0.0001174846211139946

In [152]:
mutual_info_score(df_full_train.contract, df_full_train.churn)

0.0983203874041556

In [153]:
mutual_info_score(df_full_train.partner, df_full_train.churn)

0.009967689095399745

`Partner` is more important than `Gender`, but far less important than `Contract`, as we can see from mutual information scores.

The order of importance based on mutual information scores is:
Contract, Partner, Gender

In [154]:
# applying on all categorical features to see their importance levels in relation to churn

def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [155]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

`Contract` is the most important and `Gender` is the least important categorical feature in relation to churn

### Feature importance: Correlation
For numerical columns/variables/features
- Correlation coefficient - https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

In [156]:
df_full_train[numerical]

,tenure,monthlycharges,totalcharges
0,12,19.70,258.35
1,42,73.90,3160.55
2,71,65.15,4681.75
3,71,85.45,6300.85
4,30,70.40,2044.75
...,...,...,...
5629,9,100.50,918.60
5630,60,19.95,1189.90
5631,28,105.70,2979.50
5632,2,54.40,114.10


In [157]:
df_full_train[numerical].corrwith(df_full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

The negative correlation indicates that as the tenure / total charges increases, the likelihood of churn decreases.
The positive correlation indicates that as the monthly charges increase, the likelihood of churn increases

In [158]:
# testing 

df_full_train[df_full_train.tenure <= 2].churn.mean()

np.float64(0.5953420669577875)

if someone stays with the company for 0, 1 or 2 months, the churn rate is high

In [ ]:
df_full_train[df_full_train.tenure > 2].churn.mean()

np.float64(0.22478269658378816)

if someone stays with the company for more than 2 months, the churn rate is low

In [ ]:
# between 2 months and 1 year

df_full_train[(df_full_train.tenure > 2) & (df_full_train.tenure <= 12)].churn.mean()

np.float64(0.3994413407821229)

In [ ]:
# more than 1 year

df_full_train[df_full_train.tenure > 12].churn.mean()

np.float64(0.17634908339788277)

for monthlycharges (positive correlation with respect to churn)

In [162]:
df_full_train[df_full_train.monthlycharges <= 20].churn.mean()

np.float64(0.08795411089866156)

In [163]:
df_full_train[(df_full_train.monthlycharges > 20) & (df_full_train.monthlycharges <= 50)].churn.mean()

np.float64(0.18340943683409436)

In [164]:
df_full_train[df_full_train.monthlycharges >= 50].churn.mean()

np.float64(0.3248223216635957)

In [165]:
# if we care about the importance not the direction

df_full_train[numerical].corrwith(df_full_train.churn).abs()

tenure            0.351885
monthlycharges    0.196805
totalcharges      0.196353
dtype: float64

`tenure` is more important than `monthlycharges` which is more important than `totalcharges`

## One-hot encoding
- Using Scikit-Learn to encode categorical features